<a href="https://colab.research.google.com/github/GeorgeM2000/Automatic-Keyword-Extraction-from-Documents/blob/main/Automatic_Keyword_Extraction_from_Documents.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Libraries

In [1]:
!pip install inflection

In [2]:
import nltk
import zipfile
import os
from collections import Counter, defaultdict
import re
from nltk.tokenize import RegexpTokenizer
import json
import string
from enum import Enum
from itertools import chain, groupby, product
from typing import Callable, DefaultDict, Dict, List, Optional, Set, Tuple
from sklearn.metrics import precision_recall_fscore_support
import math
from nltk.corpus import wordnet
import inflect

In [3]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

# Corpus - Corpus Preprocessing
In this section we load the corpus and examine the scientific abstracts and their corresponding keywords. We split the scientific abstracts and their keywords to a training set, a validation set, and a testing set. The training set will contain 1000 abstracts, the validation and testing set will contain 500 abstracts.

In [ ]:
!rm -rf Inspec/

In [5]:
# Path to the zip file you want to extract
zip_file_path = '/content/Inspec.zip'

# Directory where you want to extract the contents
extract_directory = '/content/'

# Create the extraction directory if it doesn't exist
os.makedirs(extract_directory, exist_ok=True)

# Open the zip file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    # Extract all contents to the specified directory
    zip_ref.extractall(extract_directory)

print("Zip file extraction complete.")

Zip file extraction complete.


In [6]:
# Directory path containing the documents
abstracts_folder_path = '/content/Inspec/docsutf8/'
document_names = []

# Loop through each file in the folder
for filename in os.listdir(abstracts_folder_path):
    file_path = os.path.join(abstracts_folder_path, filename)

    # Check if the file is a regular file (not a directory)
    if os.path.isfile(file_path):
        # Extract the name of the file without its extension
        file_name, file_extension = os.path.splitext(filename)

        try:

            # Convert the file name string to an integer
            file_name_as_int = int(file_name)
            document_names.append(file_name_as_int)

            """
            with open(file_path, 'r') as file:
                print(f"Contents of {file_name_as_int}:")
                print(file.read())
                print("------------------------------")
            """


        except ValueError:
            print(f"Filename {filename} does not represent an integer.")



#print("------------------------------")
sorted_document_names = sorted(document_names)
print(sorted_document_names)
print(document_names)

[2, 3, 4, 6, 7, 8, 9, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 102, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 

In [7]:
# Directory path containing the documents
keys_folder_path = '/content/Inspec/keys/'
document_keyword_names = []

# Loop through each file in the folder
for filename in os.listdir(keys_folder_path):
    file_path = os.path.join(keys_folder_path, filename)

    # Check if the file is a regular file (not a directory)
    if os.path.isfile(file_path):
        # Extract the name of the file without its extension
        file_name, file_extension = os.path.splitext(filename)

        try:
            # Convert the file name string to an integer
            file_name_as_int = int(file_name)
            document_keyword_names.append(file_name_as_int)

            """
            with open(file_path, 'r') as file:
                #print(f"Contents of {file_name_as_int}:")
                #print(file.read())
                #print("------------------------------")
            """

        except ValueError:
            print(f"Filename {filename} does not represent an integer.")


#print("------------------------------")
sorted_document_keyword_names = sorted(document_keyword_names)
print(sorted_document_keyword_names)
print(document_keyword_names)

[2, 3, 4, 6, 7, 8, 9, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 102, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 

In [8]:
print(f'{len(sorted_document_names)} - {len(sorted_document_keyword_names)}')

2000 - 2000


In [9]:
for index in range(len(sorted_document_names)):
  if (sorted_document_names[index] == sorted_document_keyword_names[index]) == False:
    print(f'Misalignment in the abstract {sorted_document_names[index]} and its keywords {sorted_document_keyword_names[index]}')

In [10]:
# Function to sort and rename files based on their integer filenames
def sort_files(folder_path):
    files = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]

    # Sort the files based on their filename converted to integers
    files.sort(key=lambda x: int(os.path.splitext(x)[0]))

    # Rename the files in sequential order
    for index, filename in enumerate(files, start=1):
        current_file_path = os.path.join(folder_path, filename)
        file_name, file_extension = os.path.splitext(filename)

        # Pad the index with zeros to maintain the same length of filenames
        new_filename = f"{index}{file_extension}"

        new_file_path = os.path.join(folder_path, new_filename)

        os.rename(current_file_path, new_file_path)
        print(f"File '{filename}' moved to '{new_filename}'.")

In [11]:
sort_files(abstracts_folder_path)

File '2.txt' moved to '1.txt'.
File '3.txt' moved to '2.txt'.
File '4.txt' moved to '3.txt'.
File '6.txt' moved to '4.txt'.
File '7.txt' moved to '5.txt'.
File '8.txt' moved to '6.txt'.
File '9.txt' moved to '7.txt'.
File '11.txt' moved to '8.txt'.
File '12.txt' moved to '9.txt'.
File '13.txt' moved to '10.txt'.
File '14.txt' moved to '11.txt'.
File '15.txt' moved to '12.txt'.
File '16.txt' moved to '13.txt'.
File '17.txt' moved to '14.txt'.
File '18.txt' moved to '15.txt'.
File '19.txt' moved to '16.txt'.
File '20.txt' moved to '17.txt'.
File '21.txt' moved to '18.txt'.
File '22.txt' moved to '19.txt'.
File '23.txt' moved to '20.txt'.
File '24.txt' moved to '21.txt'.
File '25.txt' moved to '22.txt'.
File '26.txt' moved to '23.txt'.
File '27.txt' moved to '24.txt'.
File '28.txt' moved to '25.txt'.
File '29.txt' moved to '26.txt'.
File '30.txt' moved to '27.txt'.
File '31.txt' moved to '28.txt'.
File '32.txt' moved to '29.txt'.
File '33.txt' moved to '30.txt'.
File '34.txt' moved to '31

In [12]:
sort_files(keys_folder_path)

File '2.key' moved to '1.key'.
File '3.key' moved to '2.key'.
File '4.key' moved to '3.key'.
File '6.key' moved to '4.key'.
File '7.key' moved to '5.key'.
File '8.key' moved to '6.key'.
File '9.key' moved to '7.key'.
File '11.key' moved to '8.key'.
File '12.key' moved to '9.key'.
File '13.key' moved to '10.key'.
File '14.key' moved to '11.key'.
File '15.key' moved to '12.key'.
File '16.key' moved to '13.key'.
File '17.key' moved to '14.key'.
File '18.key' moved to '15.key'.
File '19.key' moved to '16.key'.
File '20.key' moved to '17.key'.
File '21.key' moved to '18.key'.
File '22.key' moved to '19.key'.
File '23.key' moved to '20.key'.
File '24.key' moved to '21.key'.
File '25.key' moved to '22.key'.
File '26.key' moved to '23.key'.
File '27.key' moved to '24.key'.
File '28.key' moved to '25.key'.
File '29.key' moved to '26.key'.
File '30.key' moved to '27.key'.
File '31.key' moved to '28.key'.
File '32.key' moved to '29.key'.
File '33.key' moved to '30.key'.
File '34.key' moved to '31

In [13]:
training_set = {}
validation_set = {}
testing_set = {}

In [14]:
# Training set creation
for filename in range(1, 1501):
  abstract_file_name = f"{filename}.txt"
  keys_file_name = f"{filename}.key"
  abstract_file_path = os.path.join(abstracts_folder_path, abstract_file_name)
  keys_file_path = os.path.join(keys_folder_path, keys_file_name)

  if os.path.exists(abstract_file_path) and os.path.exists(keys_file_path):
    training_set[abstract_file_name] = keys_file_name
  else:
    print(f'{filename} is missing.')

In [ ]:
# Validation set creation (Optional)
for filename in range(1001, 1501):
  abstract_file_name = f"{filename}.txt"
  keys_file_name = f"{filename}.key"
  abstract_file_path = os.path.join(abstracts_folder_path, abstract_file_name)
  keys_file_path = os.path.join(keys_folder_path, keys_file_name)

  if os.path.exists(abstract_file_path) and os.path.exists(keys_file_path):
    validation_set[abstract_file_name] = keys_file_name
  else:
    print(f'{filename} is missing.')

# Stoplist Generation
In this section we will generate stop words to use with the automatic keyword extraction algorithms.

In [15]:
!cp -r Inspec Inspec_StopWords

In [ ]:
!rm -rf Inspec_StopWords

In [16]:
# Initialize counters
term_frequency = Counter()  # Term frequency
document_frequency = Counter()  # Document frequency
keyword_frequency = Counter()  # Keyword frequency

In [17]:
adjacent_frequency = {}  # Adjacency frequency

In [18]:
stopwords_abstracts_folder = '/content/Inspec_StopWords/docsutf8/'
stopwords_keys_folder = '/content/Inspec_StopWords/keys/'

In [19]:
abstracts_text = []
abstracts_keywords = []

In [20]:
for abstract, keys in training_set.items():

  with open(os.path.join(stopwords_abstracts_folder, abstract), 'r') as file:
    abstract_text = re.sub(r'(\n)([A-Z])', r'.\1\2', file.read())
    abstract_text = abstract_text.lower()
    abstract_text = ' '.join(abstract_text.split())
    abstracts_text.append(abstract_text)

  with open(os.path.join(stopwords_abstracts_folder, abstract), 'w') as file:
    file.write(abstract_text)


  new_keywords = []

  with open(os.path.join(stopwords_keys_folder, keys), 'r') as file:
    for line in file:
      cleaned_line = ' '.join(line.split())
      cleaned_line = cleaned_line.lower()
      new_keywords.append(cleaned_line)

  abstracts_keywords.append(new_keywords)

  with open(os.path.join(stopwords_keys_folder, keys), 'w') as file:
    for keyword in new_keywords:
      file.write(keyword + '\n')

In [21]:
# Function to process text and return word frequency
def process_text(text):
    words = re.findall(r'\b[\w-]+\b', text) #\w+
    return Counter(words)

In [22]:
# Process keywords to a set for faster lookup
keyword_term_frequency = Counter()

for keyword_file in range(1,1001):
  with open(os.path.join(stopwords_keys_folder, str(keyword_file) + '.key'), 'r', encoding='utf-8') as file:
    keywords = RegexpTokenizer(r'\b[\w-]+\b').tokenize(file.read())
    keyword_term_frequency.update(keywords)

In [23]:
# Process abstracts and calculate frequencies
for abstract, keys in training_set.items():
  with open(os.path.join(stopwords_abstracts_folder, abstract), 'r', encoding='utf-8') as file:
    abstract_text = file.read()

    tokens = RegexpTokenizer(r'\b[\w-]+\b').tokenize(abstract_text)
    distinct_words = set(tokens)

    # Calculate term frequency and document frequency
    term_frequency.update(process_text(abstract_text))
    document_frequency.update(distinct_words)


    for word in distinct_words:
      if word in keyword_term_frequency:
        # Place the corresponding keywords in a list
        keyword_frequency[word] = keyword_term_frequency[word]


In [24]:
for word in term_frequency:

  abstract_keyword_index = -1
  for abstract in abstracts_text:

    abstract_keyword_index += 1
    for line in abstracts_keywords[abstract_keyword_index]:
      if word in adjacent_frequency:
        adjacent_frequency[word] += abstract.count(word + " " + line) + abstract.count(line + " " + word)
        #adjacent_frequency[word] += abstract.count(line + " " + word)
      else:
        adjacent_frequency[word] = abstract.count(word + " " + line) + abstract.count(line + " " + word)
        #adjacent_frequency[word] = abstract.count(line + " " + word)


In [ ]:
# Read from the JSON file
with open('Adjacent_Frequency.json', 'r') as json_file:
    adjacent_frequency = json.load(json_file)

In [25]:
# Write to a JSON file
with open('Adjacent_Frequency_T1500.json', 'w') as json_file:
    json.dump(adjacent_frequency, json_file)

In [26]:
print(f'Term Frequency:{term_frequency}')
print()

print(f'Document Frequency:{document_frequency}')
print()

print(f'Keyword Frequency:{keyword_frequency}')
print()

Term Frequency:Counter({'the': 12757, 'of': 8109, 'and': 5562, 'a': 5318, 'to': 4381, 'in': 3843, 'is': 2924, 'for': 2836, 'that': 1709, 'with': 1653, 'are': 1564, 'this': 1403, 'on': 1325, 'an': 1255, 'we': 1197, 'by': 1120, 'as': 1063, 'be': 875, 'it': 784, 'system': 731, 'from': 663, 'can': 649, 'using': 614, 'which': 611, 'paper': 577, 'systems': 559, 'control': 546, 'model': 534, 'data': 517, 'method': 509, 'information': 496, 's': 495, 'or': 491, 'at': 463, 'new': 456, 'has': 447, 'have': 436, 'algorithm': 417, 'based': 405, 'results': 393, 'these': 384, 'used': 383, 'time': 371, 'problem': 367, 'was': 366, 'such': 363, 'also': 361, 'two': 348, 'not': 346, 'design': 337, 'proposed': 337, 'approach': 329, 'more': 325, 'performance': 320, 'use': 316, 'their': 311, 'its': 305, 'one': 297, 'been': 286, 'process': 286, 'between': 284, 'analysis': 280, 'our': 263, 'sub': 261, 'but': 258, 'study': 257, 'were': 257, 'some': 256, 'methods': 250, 'when': 248, 'show': 242, 'than': 239, '1':

In [27]:
stoplists = [
    [],
    [],
    [],
    [],
    [],
    []
]

In [28]:
for word, freq in term_frequency.most_common():
  if document_frequency[word] > 10:
    stoplists[0].append(word)
    if adjacent_frequency[word] > keyword_frequency[word]:
      stoplists[3].append(word)
  if document_frequency[word] > 25:
    stoplists[1].append(word)
    if adjacent_frequency[word] > keyword_frequency[word]:
      stoplists[4].append(word)
  if document_frequency[word] > 50:
    stoplists[2].append(word)
    if adjacent_frequency[word] > keyword_frequency[word]:
      stoplists[5].append(word)
  #print(f'{word}:{freq} - {document_frequency[word]} - {keyword_frequency[word]} - {adjacent_frequency[word]}')

In [29]:
for stoplist in stoplists:
  print(len(stoplist))

1830
772
351
693
353
195


# RAKE (Rapid Automatic Keyword Extraction)

## RAKE Implementation from https://csurfer.github.io/rake-nltk/_build/html/_modules/rake_nltk/rake.html

In [30]:
# Readability type definitions.
Word = str
Sentence = str
Phrase = Tuple[str, ...]

In [31]:
class Metric(Enum):
    """Different metrics that can be used for ranking."""

    DEGREE_TO_FREQUENCY_RATIO = 0  # Uses d(w)/f(w) as the metric
    WORD_DEGREE = 1  # Uses d(w) alone as the metric
    WORD_FREQUENCY = 2  # Uses f(w) alone as the metric

In [32]:
class Rake:
    """Rapid Automatic Keyword Extraction Algorithm."""

    def __init__(
        self,
        stopwords: Optional[Set[str]] = None,
        punctuations: Optional[Set[str]] = None,
        language: str = 'english',
        ranking_metric: Metric = Metric.DEGREE_TO_FREQUENCY_RATIO,
        max_length: int = 100000,
        min_length: int = 1,
        include_repeated_phrases: bool = True,
        sentence_tokenizer: Optional[Callable[[str], List[str]]] = None,
        word_tokenizer: Optional[Callable[[str], List[str]]] = None,
    ):
        """Constructor.

        :param stopwords: Words to be ignored for keyword extraction.
        :param punctuations: Punctuations to be ignored for keyword extraction.
        :param language: Language to be used for stopwords.
        :param max_length: Maximum limit on the number of words in a phrase
                           (Inclusive. Defaults to 100000)
        :param min_length: Minimum limit on the number of words in a phrase
                           (Inclusive. Defaults to 1)
        :param include_repeated_phrases: If phrases repeat in phrase list consider
                            them as is without dropping any phrases for future
                            calculations. (Defaults to True) Ex: "Magic systems is
                            a company. Magic systems was founded by Raul".

                            If repeated phrases are allowed phrase list would be
                            [
                                (magic, systems), (company,), (magic, systems),
                                (founded,), (raul,)
                            ]

                            If they aren't allowed phrase list would be
                            [
                                (magic, systems), (company,),
                                (founded,), (raul,)
                            ]
        :param sentence_tokenizer: Tokenizer used to tokenize the text string into sentences.
        :param word_tokenizer: Tokenizer used to tokenize the sentence string into words.
        """
        # By default use degree to frequency ratio as the metric.
        if isinstance(ranking_metric, Metric):
            self.metric = ranking_metric
        else:
            self.metric = Metric.DEGREE_TO_FREQUENCY_RATIO

        # If stopwords not provided we use language stopwords by default.
        self.stopwords: Set[str]
        if stopwords:
            self.stopwords = stopwords
        else:
            self.stopwords = set(nltk.corpus.stopwords.words(language))

        # If punctuations are not provided we ignore all punctuation symbols.
        self.punctuations: Set[str]
        if punctuations:
            self.punctuations = punctuations
        else:
            self.punctuations = set(string.punctuation)

        # All things which act as sentence breaks during keyword extraction.
        self.to_ignore: Set[str] = set(chain(self.stopwords, self.punctuations))

        # Assign min or max length to the attributes
        self.min_length: int = min_length
        self.max_length: int = max_length

        # Whether we should include repeated phreases in the computation or not.
        self.include_repeated_phrases: bool = include_repeated_phrases

        # Tokenizers.
        self.sentence_tokenizer: Callable[[str], List[str]]
        if sentence_tokenizer:
            self.sentence_tokenizer = sentence_tokenizer
        else:
            self.sentence_tokenizer = nltk.tokenize.sent_tokenize

        self.word_tokenizer: Callable[[str], List[str]]
        if word_tokenizer:
            self.word_tokenizer = word_tokenizer
        else:
            self.word_tokenizer = nltk.tokenize.wordpunct_tokenize

        # Stuff to be extracted from the provided text.
        self.frequency_dist: Dict[Word, int]
        self.degree: Dict[Word, int]
        self.rank_list: List[Tuple[float, Sentence]]
        self.ranked_phrases: List[Sentence]

    def extract_keywords_from_text(self, text: str):
        """Method to extract keywords from the text provided.

        :param text: Text to extract keywords from, provided as a string.
        """
        sentences: List[Sentence] = self._tokenize_text_to_sentences(text)
        self.extract_keywords_from_sentences(sentences)


    def extract_keywords_from_sentences(self, sentences: List[Sentence]):
        """Method to extract keywords from the list of sentences provided.

        :param sentences: Text to extraxt keywords from, provided as a list
                          of strings, where each string is a sentence.
        """
        phrase_list: List[Phrase] = self._generate_phrases(sentences)
        #print(phrase_list)
        self._build_frequency_dist(phrase_list)
        self._build_word_co_occurance_graph(phrase_list)
        self._build_ranklist(phrase_list)


    def get_ranked_phrases(self) -> List[Sentence]:
        """Method to fetch ranked keyword strings.

        :return: List of strings where each string represents an extracted
                 keyword string.
        """
        return self.ranked_phrases


    def get_ranked_phrases_with_scores(self) -> List[Tuple[float, Sentence]]:
        """Method to fetch ranked keyword strings along with their scores.

        :return: List of tuples where each tuple is formed of an extracted
                 keyword string and its score. Ex: (5.68, 'Four Scoures')
        """
        return self.rank_list


    def get_word_frequency_distribution(self) -> Dict[Word, int]:
        """Method to fetch the word frequency distribution in the given text.

        :return: Dictionary (defaultdict) of the format `word -> frequency`.
        """
        return self.frequency_dist


    def get_word_degrees(self) -> Dict[Word, int]:
        """Method to fetch the degree of words in the given text. Degree can be
        defined as sum of co-occurances of the word with other words in the
        given text.

        :return: Dictionary (defaultdict) of the format `word -> degree`.
        """
        return self.degree


    def _tokenize_text_to_sentences(self, text: str) -> List[Sentence]:
        """Tokenizes the given text string into sentences using the configured
        sentence tokenizer. Configuration uses `nltk.tokenize.sent_tokenize`
        by default.

        :param text: String text to tokenize into sentences.
        :return: List of sentences as per the tokenizer used.
        """
        return self.sentence_tokenizer(text)

    def _tokenize_sentence_to_words(self, sentence: Sentence) -> List[Word]:
        """Tokenizes the given sentence string into words using the configured
        word tokenizer. Configuration uses `nltk.tokenize.wordpunct_tokenize`
        by default.

        :param sentence: String sentence to tokenize into words.
        :return: List of words as per the tokenizer used.
        """
        return self.word_tokenizer(sentence)

    def _build_frequency_dist(self, phrase_list: List[Phrase]) -> None:
        """Builds frequency distribution of the words in the given body of text.

        :param phrase_list: List of List of strings where each sublist is a
                            collection of words which form a contender phrase.
        """
        self.frequency_dist = Counter(chain.from_iterable(phrase_list))

    def _build_word_co_occurance_graph(self, phrase_list: List[Phrase]) -> None:
        """Builds the co-occurance graph of words in the given body of text to
        compute degree of each word.

        :param phrase_list: List of List of strings where each sublist is a
                            collection of words which form a contender phrase.
        """
        co_occurance_graph: DefaultDict[Word, DefaultDict[Word, int]] = defaultdict(lambda: defaultdict(lambda: 0))
        for phrase in phrase_list:
            # For each phrase in the phrase list, count co-occurances of the
            # word with other words in the phrase.
            #
            # Note: Keep the co-occurances graph as is, to help facilitate its
            # use in other creative ways if required later.
            for (word, coword) in product(phrase, phrase):
                co_occurance_graph[word][coword] += 1

        # Print the co-occurance_graph
        """
        for word, co_words in co_occurance_graph.items():
            print(f"{word}:")
            for co_word, count in co_words.items():
                print(f"  {co_word}: {count}")
            print("=" * 30)
        """
        self.degree = defaultdict(lambda: 0)
        for key in co_occurance_graph:
            self.degree[key] = sum(co_occurance_graph[key].values())

    def get_plural(self, word: str):
      p = inflect.engine()
      plural = p.plural(word)
      return plural

    def _merge_word_degree_scores(self):
      merged_scores = defaultdict(int)

      for word, score in self.degree.items():
        merged_scores[word] = score
        synonyms = set()
        synonyms.add(self.get_plural(word))

        for syn in wordnet.synsets(word):
          for lemma in syn.lemmas():
            if '_' not in lemma.name():
              synonyms.add(lemma.name())
              synonyms.add(self.get_plural(lemma.name()))

        for synonym in synonyms:
          if synonym in self.degree and synonym != word:
            merged_scores[word] += self.degree[synonym]

      return merged_scores

    def _merge_word_frequency_scores(self):
      merged_scores = Counter()

      for word, score in self.frequency_dist.items():
        merged_scores[word] = score
        synonyms = set()
        synonyms.add(self.get_plural(word))

        for syn in wordnet.synsets(word):
          for lemma in syn.lemmas():
            if '_' not in lemma.name():
              synonyms.add(lemma.name())
              synonyms.add(self.get_plural(lemma.name()))

        for synonym in synonyms:
          if synonym in self.frequency_dist and synonym != word:
            merged_scores[word] += self.frequency_dist[synonym]

      return merged_scores


    def _build_ranklist(self, phrase_list: List[Phrase]):
        """Method to rank each contender phrase using the formula

              phrase_score = sum of scores of words in the phrase.
              word_score = d(w) or f(w) or d(w)/f(w) where d is degree
                           and f is frequency.

        :param phrase_list: List of List of strings where each sublist is a
                            collection of words which form a contender phrase.
        """


        #print(f'Word Degree List: {self.degree}')
        #print(f'Word Frequency List: {self.frequency_dist}')


        #self.degree = self._merge_word_degree_scores()
        #self.frequency_dist = self._merge_word_frequency_scores()

        self.rank_list = []
        for phrase in phrase_list:
            rank = 0.0
            for word in phrase:
                if self.metric == Metric.DEGREE_TO_FREQUENCY_RATIO:
                    rank += 1.0 * self.degree[word] / self.frequency_dist[word]
                elif self.metric == Metric.WORD_DEGREE:
                    rank += 1.0 * self.degree[word]
                else:
                    rank += 1.0 * self.frequency_dist[word]
            self.rank_list.append((rank, ' '.join(phrase)))
        self.rank_list.sort(reverse=True)
        self.ranked_phrases = [ph[1] for ph in self.rank_list]

    def _generate_phrases(self, sentences: List[Sentence]) -> List[Phrase]:
        """Method to generate contender phrases given the sentences of the text
        document.

        :param sentences: List of strings where each string represents a
                          sentence which forms the text.
        :return: Set of string tuples where each tuple is a collection
                 of words forming a contender phrase.
        """
        phrase_list: List[Phrase] = []
        # Create contender phrases from sentences.
        for sentence in sentences:
            word_list: List[Word] = [word.lower() for word in self._tokenize_sentence_to_words(sentence)]
            phrase_list.extend(self._get_phrase_list_from_words(word_list))

        # Based on user's choice to include or not include repeated phrases
        # we compute the phrase list and return it. If not including repeated
        # phrases, we only include the first occurance of the phrase and drop
        # the rest.
        if not self.include_repeated_phrases:
            unique_phrase_tracker: Set[Phrase] = set()
            non_repeated_phrase_list: List[Phrase] = []
            for phrase in phrase_list:
                if phrase not in unique_phrase_tracker:
                    unique_phrase_tracker.add(phrase)
                    non_repeated_phrase_list.append(phrase)
            return non_repeated_phrase_list

        return phrase_list

    def _get_phrase_list_from_words(self, word_list: List[Word]) -> List[Phrase]:
        """Method to create contender phrases from the list of words that form
        a sentence by dropping stopwords and punctuations and grouping the left
        words into phrases. Only phrases in the given length range (both limits
        inclusive) would be considered to build co-occurrence matrix. Ex:

        Sentence: Red apples, are good in flavour.
        List of words: ['red', 'apples', ",", 'are', 'good', 'in', 'flavour']
        List after dropping punctuations and stopwords.
        List of words: ['red', 'apples', *, *, good, *, 'flavour']
        List of phrases: [('red', 'apples'), ('good',), ('flavour',)]

        List of phrases with a correct length:
        For the range [1, 2]: [('red', 'apples'), ('good',), ('flavour',)]
        For the range [1, 1]: [('good',), ('flavour',)]
        For the range [2, 2]: [('red', 'apples')]

        :param word_list: List of words which form a sentence when joined in
                          the same order.
        :return: List of contender phrases honouring phrase length requirements
                 that are formed after dropping stopwords and punctuations.
        """
        groups = groupby(word_list, lambda x: x not in self.to_ignore)
        phrases: List[Phrase] = [tuple(group[1]) for group in groups if group[0]]
        return list(filter(lambda x: self.min_length <= len(x) <= self.max_length, phrases))

## RAKE Application

### Largest abstract in the corpus

In [ ]:
# Find the index and length of the paragraph with the maximum number of characters
max_length = 0
max_index = -1

for i, abstract in enumerate(abstracts_text):
    current_length = len(abstract)
    if current_length > max_length:
        max_length = current_length
        max_index = i


In [ ]:
abstracts_text[max_index]

'effect of multileaf collimator leaf width on physical dose distributions in the treatment of cns and head and neck neoplasms with intensity modulated radiation therapy the purpose of this work is to examine physical radiation dose differences between two multileaf collimator (mlc) leaf widths (5 and 10 mm) in the treatment of cns and head and neck neoplasms with intensity modulated radiation therapy (imrt). three clinical patients with cns tumors were planned with two different mlc leaf sizes, 5 and 10 mm, representing varian-120 and varian-80 millennium multileaf collimators, respectively. two sets of imrt treatment plans were developed. the goal of the first set was radiation dose conformality in three dimensions. the goal for the second set was organ avoidance of a nearby critical structure while maintaining adequate coverage of the target volume. treatment planning utilized the cadplan/helios system (varian medical systems, milpitas ca) for dynamic mlc treatment delivery. all beam

In [ ]:
max_index

990

### Abstract in "Automatic keyword extraction from individual documents" from Stuart Rose, Dave Engel, Nick Cramer and Wendy Cowley

## Analysis for one abstract

In [ ]:
abstract = ''

In [ ]:
for abstract in os.listdir(abstracts_folder_path):
  with open(os.path.join(abstracts_folder_path, abstract), 'r') as file:
    text = re.sub(r'(\n)([A-Z])', r'.\1\2', file.read())
    text = text.lower()
    text = ' '.join(text.split())

    if "compatibility of systems of linear constraints over the set of natural numbers" in text:
      abstract = text
      break

In [ ]:
abstract

'compatibility of systems of linear constraints over the set of natural numbers. criteria of compatibility of a system of linear diophantine equations, strict inequations, and nonstrict inequations are considered. upper bounds for components of a minimal set of solutions and algorithms of construction of minimal generating sets of solutions for all types of systems are given. these criteria and the corresponding algorithms for constructing a minimal supporting set of solutions can be used in solving all the considered types of systems and systems of mixed types'

In [ ]:
rake = Rake()

In [ ]:
rake.extract_keywords_from_text(abstract)

In [ ]:
ranked_keywords = rake.get_ranked_phrases_with_scores()

In [ ]:
ranked_keywords

[(8.666666666666666, 'minimal generating sets'),
 (8.5, 'linear diophantine equations'),
 (7.666666666666666, 'minimal supporting set'),
 (4.666666666666666, 'minimal set'),
 (4.5, 'linear constraints'),
 (4.0, 'upper bounds'),
 (4.0, 'strict inequations'),
 (4.0, 'nonstrict inequations'),
 (4.0, 'natural numbers'),
 (3.666666666666667, 'mixed types'),
 (3.5, 'corresponding algorithms'),
 (3.166666666666667, 'considered types'),
 (2.0, 'set'),
 (1.6666666666666667, 'types'),
 (1.5, 'considered'),
 (1.5, 'algorithms'),
 (1.0, 'used'),
 (1.0, 'systems'),
 (1.0, 'systems'),
 (1.0, 'systems'),
 (1.0, 'systems'),
 (1.0, 'system'),
 (1.0, 'solving'),
 (1.0, 'solutions'),
 (1.0, 'solutions'),
 (1.0, 'solutions'),
 (1.0, 'given'),
 (1.0, 'criteria'),
 (1.0, 'criteria'),
 (1.0, 'construction'),
 (1.0, 'constructing'),
 (1.0, 'components'),
 (1.0, 'compatibility'),
 (1.0, 'compatibility')]

In [ ]:
# Create a set to store unique keywords
unique_ranked_keywords = set()

# Filter out duplicates
filtered_keywords = []
for score, keyword in ranked_keywords:
    if keyword not in unique_ranked_keywords:
        filtered_keywords.append((score, keyword))
        unique_ranked_keywords.add(keyword)

In [ ]:
# Print the filtered list
for score, keyword in filtered_keywords:
    print(f'({score}, \'{keyword}\')')

(8.666666666666666, 'minimal generating sets')
(8.5, 'linear diophantine equations')
(7.666666666666666, 'minimal supporting set')
(4.666666666666666, 'minimal set')
(4.5, 'linear constraints')
(4.0, 'upper bounds')
(4.0, 'strict inequations')
(4.0, 'nonstrict inequations')
(4.0, 'natural numbers')
(3.666666666666667, 'mixed types')
(3.5, 'corresponding algorithms')
(3.166666666666667, 'considered types')
(2.0, 'set')
(1.6666666666666667, 'types')
(1.5, 'considered')
(1.5, 'algorithms')
(1.0, 'used')
(1.0, 'systems')
(1.0, 'system')
(1.0, 'solving')
(1.0, 'solutions')
(1.0, 'given')
(1.0, 'criteria')
(1.0, 'construction')
(1.0, 'constructing')
(1.0, 'components')
(1.0, 'compatibility')


In [ ]:
rake.get_word_degrees()

defaultdict(<function __main__.Rake._build_word_co_occurance_graph.<locals>.<lambda>()>,
            {'compatibility': 2,
             'systems': 4,
             'linear': 5,
             'constraints': 2,
             'set': 6,
             'natural': 2,
             'numbers': 2,
             'criteria': 2,
             'system': 1,
             'diophantine': 3,
             'equations': 3,
             'strict': 2,
             'inequations': 4,
             'nonstrict': 2,
             'considered': 3,
             'upper': 2,
             'bounds': 2,
             'components': 1,
             'minimal': 8,
             'solutions': 3,
             'algorithms': 3,
             'construction': 1,
             'generating': 3,
             'sets': 3,
             'types': 5,
             'given': 1,
             'corresponding': 2,
             'constructing': 1,
             'supporting': 3,
             'used': 1,
             'solving': 1,
             'mixed': 2})

In [ ]:
len(rake.get_word_degrees())

32

In [ ]:
rake.get_word_frequency_distribution()

Counter({'compatibility': 2,
         'systems': 4,
         'linear': 2,
         'constraints': 1,
         'set': 3,
         'natural': 1,
         'numbers': 1,
         'criteria': 2,
         'system': 1,
         'diophantine': 1,
         'equations': 1,
         'strict': 1,
         'inequations': 2,
         'nonstrict': 1,
         'considered': 2,
         'upper': 1,
         'bounds': 1,
         'components': 1,
         'minimal': 3,
         'solutions': 3,
         'algorithms': 2,
         'construction': 1,
         'generating': 1,
         'sets': 1,
         'types': 3,
         'given': 1,
         'corresponding': 1,
         'constructing': 1,
         'supporting': 1,
         'used': 1,
         'solving': 1,
         'mixed': 1})

In [ ]:
len(rake.get_word_frequency_distribution())

32

## Analysis for the testing set

In [33]:
# Testing set creation
for filename in range(1501, 2001):
  abstract_file_name = f"{filename}.txt"
  keys_file_name = f"{filename}.key"
  abstract_file_path = os.path.join(abstracts_folder_path, abstract_file_name)
  keys_file_path = os.path.join(keys_folder_path, keys_file_name)

  if os.path.exists(abstract_file_path) and os.path.exists(keys_file_path):
    testing_set[abstract_file_name] = keys_file_name
  else:
    print(f'{filename} is missing.')

In [34]:
for abstract, keys in testing_set.items():

  with open(os.path.join(stopwords_abstracts_folder, abstract), 'r') as file:
    abstract_text = re.sub(r'(\n)([A-Z])', r'.\1\2', file.read())
    abstract_text = abstract_text.lower()
    abstract_text = ' '.join(abstract_text.split())

  with open(os.path.join(stopwords_abstracts_folder, abstract), 'w') as file:
    file.write(abstract_text)

  new_keywords = []

  with open(os.path.join(stopwords_keys_folder, keys), 'r') as file:
    for line in file:
      cleaned_line = ' '.join(line.split())
      cleaned_line = cleaned_line.lower()
      new_keywords.append(cleaned_line)

  with open(os.path.join(stopwords_keys_folder, keys), 'w') as file:
    for keyword in new_keywords:
      file.write(keyword + '\n')

In [35]:
def read_keywords_file(keywords_file):
    with open(keywords_file, 'r', encoding='utf-8') as file:
        keywords = [line.strip() for line in file.readlines()]
    return set(keywords)

In [41]:
no_extracted_keywords = 0

total_true_positives = 0
total_no_true_keywords = 0 # Total number of true positives

stoplist_names = ['TF(df>10)','TF(df>25)','TF(df>50)','KA(df>10)','KA(df>25)','KA(KA>50)']
stoplist_names_index = 0

# Utilize each stoplist
#for stoplist in stoplists:

for abstract, keywords in testing_set.items():
    with open(os.path.join(stopwords_abstracts_folder, abstract), 'r', encoding='utf-8') as file:
      abstract_text = file.read()

    r = Rake(stopwords=stoplists[3])
    r.extract_keywords_from_text(abstract_text)
    ranked_keywords = r.get_ranked_phrases_with_scores()

    # Create a set to store unique keywords
    unique_ranked_keywords = []

    # Filter out duplicates
    for score, keyword in ranked_keywords:
        if keyword not in unique_ranked_keywords:
            unique_ranked_keywords.append(keyword)


    #T = math.floor(len(unique_ranked_keywords) * 0.5) # Parameter to limit the number of extracted keywords
    for keyword in unique_ranked_keywords:
      print(keyword)
    print()

    true_keywords = read_keywords_file(os.path.join(stopwords_keys_folder, keywords))
    print(true_keywords)
    print()

    total_no_true_keywords += len(true_keywords)
    print(total_no_true_keywords)
    print()

    no_extracted_keywords += total_no_true_keywords


    total_true_positives += len(set(unique_ranked_keywords[:T]).intersection(true_keywords))
    print(total_true_positives)
    print()
    break


# Calculate overall precision, recall, and F1-score
precision = total_true_positives / no_extracted_keywords if no_extracted_keywords > 0 else 0
recall = total_true_positives / total_no_true_keywords if total_no_true_keywords > 0 else 0
f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

#print(f'Stoplist:{stoplist_names[stoplist_names_index]}\n')
print(f'Precision: {precision}\nRecall: {recall}\nF1-Score: {f1}')
print(f'Number of extracted keywords: {no_extracted_keywords}')
print(f'Number of true keywords: {total_no_true_keywords}')
print(f'True positives: {total_true_positives}\n')

#stoplist_names_index += 1

['data mining technologies', 'data mining efforts', 'steven kudyba', 'key part', 'greater insight', 'effectively acquired', 'business productivity', 'business', 'shares', 'money', 'interview', 'information', 'field', 'expertise', 'expert', 'efficiency', 'acquisition']

{'efficiency', 'productivity', 'data mining'}

3

1

Precision: 0.3333333333333333
Recall: 0.3333333333333333
F1-Score: 0.3333333333333333
Number of extracted keywords: 3
Number of true keywords: 3
True positives: 1

